# Workflow

## Traditional: "make a change and rerun the file"

<img src="./imgs/workflow_traditional.png" width=1200px>

**Pros**
* conceptually simple
* can use any editor

**Cons**
* no interactivity (!)
* loading cost must be paid every time (!)

Note: If you want interactivity, you can keep a REPL running and use [Revise.jl](https://github.com/timholy/Revise.jl) to have changes to your script file automatically reflected in the running Julia session.

* Open `julia`
* Load Revise: `using Revise` (or put it in `~/.julia/config/startup.jl`, see [here](https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-by-default-1))
* Include your code with `includet("script.jl")` instead of `include("script.jl")`

## Visual Studio Code: IDE

<img src="./imgs/workflow_vscode.png" width=1200px>

(Image taken from https://www.julia-vscode.org/)

The [Julia extension](https://www.julia-vscode.org/) for VS Code is actively maintained.

Basic introduction: [Julia in Visual Studio Code](https://code.visualstudio.com/docs/languages/julia)

**Most important [Keybindings:](https://www.julia-vscode.org/docs/stable/userguide/keybindings/)**

* Open integrated Julia REPL: `Alt-J Alt-O`
* Kill integrated Julia REPL: `Alt-J Alt-K`
* Restart integrated Julia REPL: `Alt-J Alt-R`
* Execute a line/block of code: `Shift+Enter` and `Ctrl-Enter` (similar to Jupyter notebooks)

**Pros**
* interactivity
* lots of integrated features
  * code completion (IntelliSense)
  * documentation
  * workspace view
  * profiler
  * debugger
* (supports Jupyter notebooks as well)

**Cons**
* not as lightweight / minimal
* integrated REPL takes noticeably longer to start due to VSCodeServer / LanguageServer
* can sometimes be a bit tricky to set up on HPC clusters
  * job scheduler (SLURM, PBS) and SSH permissions
    * can we SSH to compute nodes? Under what conditions?
  * `module` software system
    * `julia` only works after, e.g., `module load julia` → wrapper script for Julia VS Code extension necessary
  * (PC2 documentation has [more information](https://uni-paderborn.atlassian.net/wiki/spaces/PC2DOK/pages/38240257/VS+Code+Remote+Usage))

# Dependencies

We're almost always using external dependencies (e.g. Julia packages, MPI, CUDA, ...). Getting the dependencies set up on a new machine/a new HPC cluster is traditionally highly non-trivial!

Julia provides a great [package manager](https://pkgdocs.julialang.org/v1/) to make this process as simple as possible.

**Most important commands:**

* `julia --project` (use the local environment)
* `] status`
* `] add SomePackage`
* `] remove SomePackage`
* `] instantiate`

## Package environments

Executability of code is only "guaranteed" if we have the **exact same versions of the dependencies** that we had when writing the code. Julia provides [package environments](https://docs.julialang.org/en/v1/manual/code-loading/#Environments-1) to deal with this.

**Project.toml**: Specifies the direct dependencies (that were `] add`ed) without versions (but, potentially, compatabilitiy bounds).

**Manifest.toml**: Auto-generated file that specifies the entire dependency tree, i.e. all dependencies with precise version information etc.

Generally speaking: Your code + `Project.toml` + `Manifest.toml` = **a reproducible project**.

### How to move code to a cluster?

Copy your reproducible project to the cluster and simply run `] instantiate` to install (and precompile) all necessary dependencies with one command.

<img src="./imgs/pkg_portability.png" width=1300px>

### Binary dependencies and Julia library (JLL) packages

[Since Julia 1.3+](https://julialang.org/blog/2019/11/artifacts/), Julia supports attaching arbitrary data to projects and packages in the form of [Julia Artifacts](https://pkgdocs.julialang.org/dev/artifacts/).

**JLL packages** (see [JuliaBinaryWrappers](https://github.com/JuliaBinaryWrappers)) are minimal wrapper packages that provide versioned binary dependencies. Just add them to your Julia environment like any other package!

Note that JLLs, like all other Julia packages, are only installed once per user and not once per project!

**Examples:** [Hwloc_jll.jl](https://github.com/JuliaBinaryWrappers/Hwloc_jll.jl) (provides the shared library `libhwloc`)

In [ ]:
using Hwloc_jll

In [ ]:
Hwloc_jll.libhwloc

In [ ]:
using Libdl

In [ ]:
dlopen(Hwloc_jll.libhwloc) do lib
    dlsym(lib, :hwloc_topology_init)
end

#### JLL production pipeline

* [BinaryBuilder](https://github.com/JuliaPackaging/BinaryBuilder.jl): Tools for building binary packages (cross compiling them in a sandbox environment)
* [Yggdrasil](https://github.com/JuliaPackaging/Yggdrasil): Community buildtree of BinaryBuilder recipes for building binary packages.
* [JuliaBinaryWrappers](https://github.com/JuliaBinaryWrappers): Final Julia packages that wrap binary dependencies.

**Pipeline:** [BinaryBuilder](https://github.com/JuliaPackaging/BinaryBuilder.jl) → [Yggdrasil](https://github.com/JuliaPackaging/Yggdrasil) → [JuliaBinaryWrappers](https://github.com/JuliaBinaryWrappers)

#### Using pre-installed system binaries (if you need to)

One can use [Julia preferences](https://docs.julialang.org/en/v1/manual/code-loading/#preferences) to teach wrapper packages like MPI.jl or CUDA.jl to use a system installation of MPI or CUDA instead of JLLs.

**MPI.jl**

```julia
using MPIPreferences
MPIPreferences.use_system_binary()
```

**CUDA.jl**

```julia
using CUDA
CUDA.set_runtime_version!(v"12.2"; local_toolkit=true)
```

Note, however, that JLLs and their binaries are set up in such a way that they work nicely together. If you use system binaries the burden of ensuring this is on you (or the HPC admins).